In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import gc

In [2]:
tracks = pd.read_csv('tracks.csv')
tracks.columns = tracks.iloc[0]
tracks = tracks.drop([0, 1])
new_cols = tracks.columns.values
new_cols[0] = 'track_id'
tracks.columns = new_cols
tracks = tracks.reset_index(drop=True)

/home/daniel/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
mfcc = pickle.load(open('mel-spec.pkl', 'rb'))
gc.collect()
mfcc[0]

array([[-65.14118 , -51.263527, -47.838448, ..., -41.815773, -37.481846,
        -25.274061],
       [-65.48186 , -54.84303 , -52.83989 , ..., -16.392946, -16.717213,
        -17.64329 ],
       [-69.560745, -62.088215, -56.7553  , ..., -16.151316, -15.893209,
        -18.886696],
       ...,
       [-79.502   , -72.30862 , -59.821915, ..., -42.443546, -45.269314,
        -50.11664 ],
       [-80.      , -78.63363 , -63.498665, ..., -46.985542, -50.077003,
        -55.21499 ],
       [-80.      , -80.      , -73.89963 , ..., -56.54886 , -58.29077 ,
        -60.552216]], dtype=float32)

In [4]:
mfcc_df = pd.DataFrame(mfcc)
index_nan = np.argwhere(pd.isnull(mfcc))

In [9]:
mfcc_df = mfcc_df.dropna()
mfcc = mfcc_df.to_numpy().ravel()

for i in range(len(mfcc)):
    mfcc[i] = np.append(mfcc[i], np.zeros((128, 1293 - mfcc[i].shape[1])), axis=1)
    
index_nan = index_nan.ravel()

AttributeError: 'DataFrame' object has no attribute 'to_list'

In [8]:
mfcc[0].shape

(128, 1291)

In [ ]:
del mfcc_df

In [ ]:
gc.collect()

In [ ]:
tracks = tracks.loc[tracks['subset'] == 'small']
tracks = tracks.reset_index()
tracks = tracks.drop(index_nan)
tracks = tracks.reset_index()

In [ ]:
y_train = tracks.loc[(tracks['split'] == 'training')]['genre_top']
y_val = tracks.loc[(tracks['split'] == 'validation')]['genre_top']
y_test = tracks.loc[(tracks['split'] == 'test')]['genre_top']

In [ ]:
index_train = y_train.index
index_val = y_val.index
index_test = y_test.index
print(index_train)
print(index_val)
print(index_test)

In [ ]:
len(y_train) + len(y_val) + len(y_test)

In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)
y_test = le.transform(y_test)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=8, dtype='int32')
y_val = tf.keras.utils.to_categorical(y_val, num_classes=8, dtype='int32')
y_test = tf.keras.utils.to_categorical(y_test, num_classes=8, dtype='int32')

gc.collect()

In [ ]:
mfcc = np.stack(mfcc.ravel()).reshape(-1, 20, 1293)
gc.collect()

In [ ]:
mfcc.shape

In [ ]:
mfcc = mfcc.reshape((-1, 20, 1293, 1))
mfcc.shape

In [ ]:
mfcc[0].shape

In [ ]:
X_train = mfcc[index_train]
X_val = mfcc[index_val]
X_test = mfcc[index_test]

In [ ]:
len(X_train)

In [ ]:
y_train.shape

In [ ]:
activation='relu'
optimizer = tf.keras.optimizers.Adam()
metrics=['accuracy']

model = tf.keras.Sequential()
model.add(Conv2D(64, kernel_size=3, kernel_regularizer=tf.keras.regularizers.l2(), strides=1, activation=activation, input_shape=(20, 1293, 1)))
model.add(MaxPool2D(pool_size=(2,4)))
model.add(Conv2D(64, kernel_size=(3,5), kernel_regularizer=tf.keras.regularizers.l2(), strides=1, activation=activation))
model.add(MaxPool2D(pool_size=(2,4)))
model.add(Dense(32, kernel_regularizer=tf.keras.regularizers.l2(), activation=activation))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(8, activation='softmax'))

model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=metrics)
mcp_save = ModelCheckpoint('best_model_cnn.hdf5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)
history = model.fit(X_train, y_train, validation_split=0.1, epochs=100, verbose=1, callbacks=[mcp_save])
plt.plot(history.history['' + metrics[0]])
plt.plot(history.history['val_accuracy'])
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

best_model = load_model('best_model_cnn.hdf5')
print("Train:")
print(best_model.evaluate(X_train, y_train, verbose=2))
print("Val:")
print(best_model.evaluate(X_val, y_val, verbose=2))